In [ ]:
import yfinance as yf
!pip install ta
from ta import add_all_ta_features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    ConfusionMatrixDisplay
)
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Using Apple (AAPL) as our example stock
ticker = 'AAPL'
period = '2y'

print(f"Fetching data for {ticker}...")
data = yf.download(ticker, period=period, interval='1d', progress=False)

# Get market context (S&P 500 and VIX)
spx = yf.download("^GSPC", period=period, interval='1d', progress=False)['Close']
vix = yf.download("^VIX", period=period, interval='1d', progress=False)['Close']

if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

if len(data.columns) == 5:
    data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

print(f"Downloaded {len(data)} days of data")

In [ ]:
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    if isinstance(data[col].iloc[0], (np.ndarray, list)):
        data[col] = data[col].apply(lambda x: x[0])

data = add_all_ta_features(
    data, 
    open="Open", 
    high="High", 
    low="Low", 
    close="Close", 
    volume="Volume",
    fillna=True
)

In [ ]:
# Multi-period returns
data['Return_1d'] = data['Close'].pct_change()
data['Return_3d'] = data['Close'].pct_change(3)
data['Return_5d'] = data['Close'].pct_change(5)
data['Return_10d'] = data['Close'].pct_change(10)

# Moving averages
data['MA_5'] = data['Close'].rolling(5).mean()
data['MA_10'] = data['Close'].rolling(10).mean()
data['MA_20'] = data['Close'].rolling(20).mean()
data['MA_50'] = data['Close'].rolling(50).mean()

# Volatility regimes
data['Vol_5d'] = data['Return_1d'].rolling(5).std()
data['Vol_10d'] = data['Return_1d'].rolling(10).std()
data['Vol_20d'] = data['Return_1d'].rolling(20).std()
data['Volatility_Regime'] = (data['Vol_10d'] > data['Vol_10d'].median()).astype(int)

# Price relative features
data['Close_to_High'] = data['Close'] / data['High']
data['Close_to_Low'] = data['Close'] / data['Low']
data['High_Low_Range'] = (data['High'] - data['Low']) / data['Close']

# Volume analysis
data['Volume_Change'] = data['Volume'].pct_change()
data['Vol_MA_5'] = data['Volume'].rolling(5).mean()
data['Vol_MA_20'] = data['Volume'].rolling(20).mean()
data['Volume_Ratio'] = data['Volume'] / data['Vol_MA_20']
data['Vol_Spike'] = (data['Volume'] / data['Volume'].rolling(10).mean() > 1.5).astype(int)

# Price gaps
data['Gap'] = (data['Open'] - data['Close'].shift(1)) / data['Close'].shift(1)
data['Gap_Up'] = (data['Gap'] > 0.005).astype(int)
data['Gap_Down'] = (data['Gap'] < -0.005).astype(int)

# MA crossovers
data['MA5_MA20_Cross'] = (data['MA_5'] > data['MA_20']).astype(int)
data['MA10_MA50_Cross'] = (data['MA_10'] > data['MA_50']).astype(int)
data['Price_Above_MA50'] = (data['Close'] > data['MA_50']).astype(int)

# Momentum indicators
data['Momentum_3d'] = data['Close'] / data['Close'].shift(3) - 1
data['Momentum_5d'] = data['Close'] / data['Close'].shift(5) - 1
data['Momentum_10d'] = data['Close'] / data['Close'].shift(10) - 1

# Candlestick patterns
data['Bullish_Engulfing'] = (
    (data['Close'] > data['Open']) & 
    (data['Close'].shift(1) < data['Open'].shift(1)) &
    (data['Close'] > data['Open'].shift(1)) &
    (data['Open'] < data['Close'].shift(1))
).astype(int)

data['Bearish_Engulfing'] = (
    (data['Close'] < data['Open']) & 
    (data['Close'].shift(1) > data['Open'].shift(1)) &
    (data['Close'] < data['Open'].shift(1)) &
    (data['Open'] > data['Close'].shift(1))
).astype(int)

data['Doji'] = (
    abs(data['Close'] - data['Open']) / (data['High'] - data['Low']) < 0.1
).astype(int)

# External market context - align indices first
spx_aligned = spx.reindex(data.index, method='ffill')
vix_aligned = vix.reindex(data.index, method='ffill')

data['SPX_Return'] = spx_aligned.pct_change()
spx_ma_5 = spx_aligned.rolling(5).mean()
data['SPX_MA_5'] = spx_ma_5
data['Market_Trend'] = (spx_aligned > spx_ma_5).astype(int)
data['VIX'] = vix_aligned
vix_ma_20 = vix_aligned.rolling(20).mean()
data['VIX_High'] = (vix_aligned > vix_ma_20).astype(int)

# Lag features for temporal patterns
lag_features = ['Return_1d', 'momentum_rsi', 'trend_macd', 'Volume', 'Vol_10d']
for feat in lag_features:
    if feat in data.columns:
        for lag in [1, 2, 3, 5]:
            data[f'{feat}_lag{lag}'] = data[feat].shift(lag)

# Price position indicators
if 'trend_sma_fast' in data.columns:
    data['Price_vs_SMA20'] = data['Close'] / data['trend_sma_fast']
if 'trend_sma_slow' in data.columns:
    data['Price_vs_SMA50'] = data['Close'] / data['trend_sma_slow']

# Bollinger Bands position
if 'volatility_bbh' in data.columns and 'volatility_bbl' in data.columns:
    bb_width = data['volatility_bbh'] - data['volatility_bbl']
    data['BB_Position'] = (data['Close'] - data['volatility_bbl']) / bb_width
    data['BB_Squeeze'] = (bb_width / data['Close'] < 0.02).astype(int)

# RSI levels
if 'momentum_rsi' in data.columns:
    data['RSI_Overbought'] = (data['momentum_rsi'] > 70).astype(int)
    data['RSI_Oversold'] = (data['momentum_rsi'] < 30).astype(int)
    data['RSI_Neutral'] = ((data['momentum_rsi'] >= 45) & (data['momentum_rsi'] <= 55)).astype(int)

data.dropna(inplace=True)
print(f"Feature engineering complete: {data.shape[1]} features")

In [ ]:
data['Future_Return_3d'] = data['Close'].shift(-3) / data['Close'] - 1
data['Target'] = np.where(
    data['Future_Return_3d'] > 0.01, 1,
    np.where(data['Future_Return_3d'] < -0.01, 0, np.nan)
)
data.dropna(inplace=True)

print(f"\nTarget distribution (3-day return > 1%):")
print(data['Target'].value_counts())
print(f"Final dataset: {len(data)} samples")

In [ ]:
exclude_cols = ['Target', 'Adj Close', 'Future_Return_3d']
feature_cols = [col for col in data.columns if col not in exclude_cols]

X = data[feature_cols]
y = data['Target']

# Remove low variance features
selector = VarianceThreshold(threshold=0.001)
X_selected = selector.fit_transform(X)
selected_features = X.columns[selector.get_support()].tolist()

print(f"After variance threshold: {len(selected_features)} features")

# Remove highly correlated features
X_temp = pd.DataFrame(X_selected, columns=selected_features)
corr_matrix = X_temp.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.9)]
X_temp.drop(columns=to_drop, inplace=True)

print(f"After correlation filter: {X_temp.shape[1]} features")

# Feature importance ranking with Random Forest
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=5)
rf_selector.fit(X_temp, y)
importances = pd.Series(rf_selector.feature_importances_, index=X_temp.columns)
top_features = importances.nlargest(40).index.tolist()

X_final = X_temp[top_features]
print(f"Using top 40 features based on RF importance\n")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

In [ ]:
# Walk-forward validation with 5 splits
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = []

print("=" * 70)
print("WALK-FORWARD CROSS-VALIDATION")
print("=" * 70)

fold = 1
for train_idx, val_idx in tscv.split(X_scaled):
    X_train_cv, X_val_cv = X_scaled[train_idx], X_scaled[val_idx]
    y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
    
    # Quick XGBoost validation
    xgb_cv = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    xgb_cv.fit(X_train_cv, y_train_cv)
    val_pred = xgb_cv.predict(X_val_cv)
    val_acc = accuracy_score(y_val_cv, val_pred)
    cv_scores.append(val_acc)
    
    print(f"Fold {fold}: Val Accuracy = {val_acc:.4f}")
    fold += 1

print(f"\nMean CV Accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
print("=" * 70 + "\n")


In [ ]:
split_idx = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}\n")

# Class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}

# Add Gaussian noise for regularization
noise = np.random.normal(0, 0.01, X_train.shape)
X_train_noisy = X_train + noise


In [ ]:
print("Training XGBoost model")
xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss',
    random_state=42
)
xgb_model.fit(X_train_noisy, y_train)

In [ ]:
print("Training LightGBM model")
lgb_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train_noisy, y_train)

In [ ]:
print("Building neural network")
nn_model = Sequential([
    Dense(128, input_dim=X_train.shape[1], kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

nn_model.compile(
    optimizer=Adam(learning_rate=0.001, decay=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=0
)

lr_reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=7,
    min_lr=1e-6,
    verbose=0
)

print("Training neural network")
history = nn_model.fit(
    X_train_noisy, y_train,
    epochs=150,
    batch_size=16,
    validation_split=0.15,
    class_weight=class_weight_dict,
    callbacks=[early_stop, lr_reduce],
    verbose=0
)

In [ ]:
print("Building stacking ensemble")
stack_model = StackingClassifier(
    estimators=[
        ('xgb', xgb_model),
        ('lgb', lgb_model),
        ('rf', RandomForestClassifier(n_estimators=200, max_depth=6, random_state=42))
    ],
    final_estimator=LogisticRegression(),
    cv=3,
    n_jobs=-1
)
stack_model.fit(X_train, y_train)

In [ ]:
print("\n" + "=" * 70)
print("MODEL PERFORMANCE COMPARISON")
print("=" * 70)

models = {
    'XGBoost': xgb_model,
    'LightGBM': lgb_model,
    'Neural Network': nn_model,
    'Stacking Ensemble': stack_model
}

results = {}
for name, model in models.items():
    if name == 'Neural Network':
        y_pred_proba = model.predict(X_test, verbose=0)
        y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    else:
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    try:
        auc = roc_auc_score(y_test, y_pred_proba)
    except:
        auc = 0.0
    
    results[name] = {'accuracy': acc, 'roc_auc': auc, 'predictions': y_pred}
    
    print(f"\n{name}:")
    print(f"  Accuracy: {acc:.4f} ({acc*100:.2f}%)")
    print(f"  ROC-AUC:  {auc:.4f}")

# Find best model
best_model_name = max(results, key=lambda x: results[x]['roc_auc'])
best_predictions = results[best_model_name]['predictions']

print(f"\n{'=' * 70}")
print(f"BEST MODEL: {best_model_name}")
print(f"{'=' * 70}\n")

# Detailed report for best model
print(classification_report(y_test, best_predictions, target_names=['DOWN', 'UP']))

# Confusion matrix
cm = confusion_matrix(y_test, best_predictions)
disp = ConfusionMatrixDisplay(cm, display_labels=['DOWN', 'UP'])
disp.plot(cmap='Blues', values_format='d')
plt.title(f'Confusion Matrix - {best_model_name}')
plt.tight_layout()
plt.show()

In [ ]:
print("=" * 70)
print("CLASS DISTRIBUTION ANALYSIS")
print("=" * 70)
test_data = data.iloc[split_idx:split_idx + len(y_test)].copy()

# Check training distribution
train_dist = y_train.value_counts()
print("\nTraining Set:")
print(train_dist)
print(f"DOWN: {train_dist[0]/len(y_train)*100:.1f}%")
print(f"UP:   {train_dist[1]/len(y_train)*100:.1f}%")

# Check test distribution
test_dist = y_test.value_counts()
print("\nTest Set:")
print(test_dist)
print(f"DOWN: {test_dist[0]/len(y_test)*100:.1f}%")
print(f"UP:   {test_dist[1]/len(y_test)*100:.1f}%")


print("\n" + "=" * 70)
print("PREDICTION ANALYSIS")
print("=" * 70)

# Get probabilities from LightGBM (your best model)
y_pred_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]
test_data['Prediction'] = (y_pred_proba_lgb > 0.086).astype(int)  # ← OPTIMAL THRESHOLD

test_data['Actual_Return_3d'] = test_data['Future_Return_3d']
test_data['Strategy_Return'] = np.where(
    test_data['Prediction'] == 1,
    test_data['Actual_Return_3d'],
    0
)

# What is the model actually predicting?
pred_dist = pd.Series(y_pred_default).value_counts()
print("\nDefault Predictions (threshold=0.5):")
print(pred_dist)
print(f"Predicting DOWN: {pred_dist.get(0, 0)/len(y_pred_default)*100:.1f}%")
print(f"Predicting UP:   {pred_dist.get(1, 0)/len(y_pred_default)*100:.1f}%")

# Check probability distribution
print(f"\nProbability Statistics:")
print(f"Mean probability: {y_pred_proba.mean():.3f}")
print(f"Median probability: {np.median(y_pred_proba):.3f}")
print(f"Probabilities > 0.5: {(y_pred_proba > 0.5).sum()} ({(y_pred_proba > 0.5).mean()*100:.1f}%)")

#FIND OPTIMAL THRESHOLD

print("\n" + "=" * 70)
print("THRESHOLD OPTIMIZATION")
print("=" * 70)

# Calculate precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba)

# Calculate F1 scores for each threshold
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1] + 1e-10)

# Find optimal threshold (maximizes F1)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

print(f"\nOptimal threshold: {optimal_threshold:.3f}")
print(f"This maximizes F1 score: {f1_scores[optimal_idx]:.3f}")

# Try multiple thresholds
print("\nPerformance at different thresholds:")
print("-" * 70)
print(f"{'Threshold':<12} {'Accuracy':<12} {'F1 Score':<12} {'UP Recall':<12}")
print("-" * 70)

for threshold in [0.3, 0.4, optimal_threshold, 0.5, 0.6]:
    y_pred_thresh = (y_pred_proba > threshold).astype(int)
    acc = (y_pred_thresh == y_test).mean()
    f1 = f1_score(y_test, y_pred_thresh)
    
    # Calculate recall for UP class specifically
    up_mask = y_test == 1
    up_recall = (y_pred_thresh[up_mask] == 1).mean() if up_mask.sum() > 0 else 0
    
    marker = " ← OPTIMAL" if abs(threshold - optimal_threshold) < 0.01 else ""
    print(f"{threshold:<12.3f} {acc:<12.3f} {f1:<12.3f} {up_recall:<12.3f}{marker}")

# ============================================================================
# STEP 4: APPLY OPTIMAL THRESHOLD
# ============================================================================

print("\n" + "=" * 70)
print("RESULTS WITH OPTIMAL THRESHOLD")
print("=" * 70)

y_pred_optimal = (y_pred_proba > optimal_threshold).astype(int)

print(f"\nClassification Report (threshold = {optimal_threshold:.3f}):")
print(classification_report(y_test, y_pred_optimal, target_names=['DOWN', 'UP']))

# Compare to default
print(f"\nClassification Report (default threshold = 0.086):")
print(classification_report(y_test, y_pred_default, target_names=['DOWN', 'UP']))

# ============================================================================
# STEP 5: VISUALIZE THE IMPROVEMENT
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Probability Distribution
axes[0, 0].hist(y_pred_proba[y_test == 0], bins=30, alpha=0.6, label='Actual DOWN', color='red')
axes[0, 0].hist(y_pred_proba[y_test == 1], bins=30, alpha=0.6, label='Actual UP', color='green')
axes[0, 0].axvline(0.5, color='blue', linestyle='--', linewidth=2, label='Default (0.5)')
axes[0, 0].axvline(optimal_threshold, color='orange', linestyle='--', linewidth=2, label=f'Optimal ({optimal_threshold:.2f})')
axes[0, 0].set_xlabel('Predicted Probability')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Prediction Probability Distribution')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Precision-Recall Curve
axes[0, 1].plot(recalls[:-1], precisions[:-1], linewidth=2, color='blue')
axes[0, 1].scatter(recalls[optimal_idx], precisions[optimal_idx], 
                   s=200, c='red', marker='*', zorder=5, label=f'Optimal (F1={f1_scores[optimal_idx]:.3f})')
axes[0, 1].set_xlabel('Recall')
axes[0, 1].set_ylabel('Precision')
axes[0, 1].set_title('Precision-Recall Curve')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. F1 Score vs Threshold
axes[0, 2].plot(thresholds, f1_scores, linewidth=2, color='green')
axes[0, 2].axvline(optimal_threshold, color='red', linestyle='--', linewidth=2)
axes[0, 2].set_xlabel('Threshold')
axes[0, 2].set_ylabel('F1 Score')
axes[0, 2].set_title('F1 Score vs Threshold')
axes[0, 2].grid(alpha=0.3)

# 4. Confusion Matrix - Default
cm_default = confusion_matrix(y_test, y_pred_default)
sns.heatmap(cm_default, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['DOWN', 'UP'], yticklabels=['DOWN', 'UP'],
            ax=axes[1, 0])
axes[1, 0].set_title('Confusion Matrix - Default (0.5)')
axes[1, 0].set_ylabel('True Label')
axes[1, 0].set_xlabel('Predicted Label')

# 5. Confusion Matrix - Optimal
cm_optimal = confusion_matrix(y_test, y_pred_optimal)
sns.heatmap(cm_optimal, annot=True, fmt='d', cmap='Greens',
            xticklabels=['DOWN', 'UP'], yticklabels=['DOWN', 'UP'],
            ax=axes[1, 1])
axes[1, 1].set_title(f'Confusion Matrix - Optimal ({optimal_threshold:.2f})')
axes[1, 1].set_ylabel('True Label')
axes[1, 1].set_xlabel('Predicted Label')

# 6. Metrics Comparison
metrics = {
    'Accuracy': [
        (y_pred_default == y_test).mean(),
        (y_pred_optimal == y_test).mean()
    ],
    'F1 Score': [
        f1_score(y_test, y_pred_default),
        f1_score(y_test, y_pred_optimal)
    ],
    'UP Recall': [
        (y_pred_default[y_test == 1] == 1).mean(),
        (y_pred_optimal[y_test == 1] == 1).mean()
    ]
}

x = np.arange(len(metrics))
width = 0.35
bars1 = axes[1, 2].bar(x - width/2, [metrics[m][0] for m in metrics], width, label='Default (0.5)', color='lightblue')
bars2 = axes[1, 2].bar(x + width/2, [metrics[m][1] for m in metrics], width, label=f'Optimal ({optimal_threshold:.2f})', color='lightgreen')

axes[1, 2].set_ylabel('Score')
axes[1, 2].set_title('Metric Comparison')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(metrics.keys())
axes[1, 2].legend()
axes[1, 2].grid(alpha=0.3, axis='y')
axes[1, 2].set_ylim([0, 1])

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[1, 2].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)

In [ ]:
print("\n" + "=" * 70)
print("TRADING STRATEGY SIMULATION")
print("=" * 70)

# Align predictions with test data
test_data = data.iloc[split_idx:split_idx + len(y_test)]
test_data = test_data.copy()
test_data['Prediction'] = best_predictions
test_data['Actual_Return'] = test_data['Close'].pct_change()

# Strategy: Go long when predicting UP (1), flat otherwise
test_data['Strategy_Return'] = np.where(
    test_data['Prediction'] == 1,
    test_data['Actual_Return'],
    0
)

# Calculate cumulative returns
cumulative_market = (1 + test_data['Actual_Return']).cumprod() - 1
cumulative_strategy = (1 + test_data['Strategy_Return']).cumprod() - 1

print(f"\nBuy & Hold Return: {cumulative_market.iloc[-1]:.2%}")
print(f"Strategy Return:   {cumulative_strategy.iloc[-1]:.2%}")
print(f"Outperformance:    {(cumulative_strategy.iloc[-1] - cumulative_market.iloc[-1]):.2%}%")

# Plot cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(cumulative_market.values, label='Buy & Hold', linewidth=2)
plt.plot(cumulative_strategy.values, label='ML Strategy', linewidth=2)
plt.xlabel('Trading Days')
plt.ylabel('Cumulative Return')
plt.title(f'Strategy Performance - {best_model_name}')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Feature importance (from best tree model)
if best_model_name in ['XGBoost', 'LightGBM']:
    best_tree_model = models[best_model_name]
    feat_imp = pd.Series(
        best_tree_model.feature_importances_,
        index=X_final.columns
    ).nlargest(15)
    
    plt.figure(figsize=(10, 6))
    feat_imp.plot(kind='barh')
    plt.xlabel('Importance')
    plt.title(f'Top 15 Features - {best_model_name}')
    plt.tight_layout()
    plt.show()

print("\n" + "=" * 70)
print("ANALYSIS COMPLETE")
print("=" * 70)